In [6]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def fit_full_training_data(hyperparameter_setting, cv_file, 
                         generator_params_dict, history_path,
                         setting_count, original_model_path,
                         model_name):
        
        cv_setting = load_cross_validation_settings(cv_file)
        train = cv_setting['Train']
        
        cur_train = train[1]

        final_train = cur_train['Train']
        final_validation = cur_train['Validation']
        
        subject_dict_train = {}
        subject_list_train = []
        for x,y in zip(final_train[0], final_train[1]):
            subject_list_train.append(x)
            subject_dict_train[x] = y
            
        subject_dict_val = {}
        subject_list_val = []
        for x,y in zip(final_validation[0], final_validation[1]):
            subject_list_val.append(x)
            subject_dict_val[x] = y
        
        temp_dict = subject_dict_train.copy()
        temp_dict.update(subject_dict_val)
        full_subject_list = subject_list_train + subject_list_val
        
        print(full_subject_list)
        
        import sys
        sys.exit(1)
        
        training_generator = DataGenerator(subject_list_val, temp_dict, **generator_params_dict)

        for i,j in training_generator:
            print(i.shape,j.shape)
        history = fit_generator(hyperparameter_setting, training_generator, 
                                model, setting_count, 1,
                               False)
        
        model.save(model_name)
        backend.clear_session()

                
       
    
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/history'
sampler_file1 = r'DTE_ADNI_Baseline_Axial_CN_vs_AD_First10_Iters.pkl'
sampler_file2 = r'DTE_ADNI_Baseline_Axial_CN_vs_AD_Second10_Iters.pkl'
checkpoint_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/model_checkpoints'
base_save_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_CN_vs_AD_CV_File.pickle'

hyperparam_list1 = load_hyperparameter_settings(sampler_file1)
hyperparam_list2 = load_hyperparameter_settings(sampler_file2)
full_hyperparams = hyperparam_list1 + hyperparam_list2

selected_hyperparams = [10,2,16,1,4,6,17,15,13] #full params are from 1 to 20

for count,idx in enumerate(selected_hyperparams):
    hyperparam = full_hyperparams[idx - 1]

    if idx <= 10:
        model = load_model('/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/original_models/'+str(idx)+'.hdf5')
    else:
        model = load_model('/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/original_models/Second10_Iters/'+str(idx)+'.hdf5')

    #print(model.summary())
    
    params_dict = {'dim':(img_width,img_height),
                   'batch_size': hyperparam['batch_size'],
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
    
    model_name = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_' + str(count + 1) + '.hdf5'
    fit_full_training_data(hyperparam, cv_file, params_dict, history_path,
                         count, '', model_name)

['ADNI_100_S_1286_MR_MPR____N3__Scaled_Br_20070808200522693_S27102_I66144_brain.mgz_123.npy', 'ADNI_012_S_0689_MR_MPR-R____N3__Scaled_2_Br_20081001123144447_S16392_I118713_brain.mgz_114.npy', 'ADNI_037_S_0627_MR_MPR__GradWarp__N3__Scaled_Br_20071030172722998_S16313_I79831_brain.mgz_132.npy', 'ADNI_100_S_0747_MR_MPR-R____N3__Scaled_Br_20080207103116048_S18394_I89761_brain.mgz_107.npy', 'ADNI_041_S_0125_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070109234918469_S10883_I35671_brain.mgz_141.npy', 'ADNI_126_S_0680_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070209193929552_S16099_I38926_brain.mgz_116.npy', 'ADNI_131_S_0123_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070801185420724_S10960_I63784_brain.mgz_137.npy', 'ADNI_006_S_0731_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080220094836683_S18321_I90848_brain.mgz_134.npy', 'ADNI_005_S_0221_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070910164745994_S11958_I72128_brain.mgz_119.npy', 'ADNI_130_S_1200_MR_MPR-R__GradW

SystemExit: 1

/home/iitindmaths/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
final_train

NameError: name 'final_train' is not defined